# A Gospel in the Language of Lies
### Interactive Figures and Data Appendix

In [ ]:
# import statements
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from ipywidgets import interactive, fixed
import os

In [ ]:
# only necessary in colab, but won't hurt
if not os.path.exists('output'):
    os.mkdir('output')

#### Download the data from GitHub

If spreadsheets are more your style, you can also [import this CSV](https://github.com/cwf2/homerocentones/blob/main/data/leave_one_out.csv) directly into Excel.

In [ ]:
data_path = 'https://raw.githubusercontent.com/cwf2/homerocentones/main/data/leave_one_out.csv'
df = pd.read_csv(data_path, na_values=[''], keep_default_na=False)

# relabel lines of unknown source
df['hom_work'] = df['hom_work'].replace('0', np.NaN)
df['hom_spkr_being'] = df['hom_spkr_being'].replace('0', np.NaN)

# make deception tags ordered categorical values
df['dec_line'] = pd.Categorical(df['dec_line'],
    categories = ['unclassified', 'None', 'Possible', 'Strong'], ordered=True)
df['dec_speech'] = pd.Categorical(df['dec_speech'],
    categories = ['unclassified', 'None', 'Possible', 'Strong'], ordered=True)

#### General style settings

This creates a uniform look for the graphs.

In [ ]:
sns.set_style('white')

#### Source lines for Eudocia by work

A little more than half of Eudocia's source lines come from the *Odyssey*, with fewer coming from the *Iliad*.

Note: In the pie charts below, we have counted lines that occur more than once in Homer proportionally: if the same line occurs, for example, twice in the *Odyssey* and once in the *Iliad*, then each occurrence will be counted as one-third of a line, so that the total for *Odyssey* is 0.67 and that for the *Iliad* is 0.33.

In [ ]:
# summary table

lines_by_work = df.groupby('hom_work').agg(
    lines = ('coefficient.1', 'sum'),
)

display(lines_by_work)

In [ ]:
# pie chart
fig, ax = plt.subplots()

wedges, texts, autotexts = ax.pie(
    x = lines_by_work['lines'], 
    labels=['Iliad', 'Odyssey'], 
    labeldistance=None,
    autopct='%d%%',
)

ax.legend(loc='center left', bbox_to_anchor=(1, 0, 0.5, 1))
ax.set_title('Homeric sources for Eudocia’s lines (speeches only)')
plt.setp(autotexts, size=14, weight="bold", color="white")

# save figure as PDF
output_file = os.path.join('output', 'fig_source_by_work.pdf')
plt.savefig(output_file)

# display
plt.show()

#### By Homeric source speaker 

Here we create a simplified set of speakers in which the four characters most used by Eudocia (Odysseus, Achilles, Telemachus and Penelope) are distinguished, while the remaining mortal characters are grouped together for simplicity, as are all divine speakers. Narrator text is also kept separate.

In [ ]:
# simplified list of characters
spkrs = df['hom_spkr_being'].copy()
spkrs[spkrs=='mortal'] = 'other mortals'
spkrs[spkrs=='divine'] = 'gods'
spkrs[spkrs=='Narrator'] = 'narrator'

for name in ['Odysseus', 'Achilles', 'Telemachus', 'Penelope',]:
    mask = df['hom_spkr'].str.startswith(name)
    spkrs[mask] = name

In [ ]:
# summary table
lines_by_hom_spkr = df.groupby(spkrs).agg(
    lines = ('coefficient.1', 'sum'),
)

# reorder
order = ['Odysseus', 'Achilles', 'Telemachus', 'Penelope', 'narrator', 'other mortals', 'gods', 'other']
lines_by_hom_spkr = lines_by_hom_spkr.loc[order, :]

display(lines_by_hom_spkr)

In [ ]:
fig, ax = plt.subplots()
wedges, texts, autotexts = ax.pie(
    x = lines_by_hom_spkr['lines'],
    labels = order,
    counterclock = False,
    startangle = 90,
    autopct='%1.f%%',
)
ax.set_title('Eudocia’s use of Homeric speakers')
plt.setp(autotexts, size=10, weight="bold", color="white")

# save figure as PDF
output_file = os.path.join('output', 'fig_source_by_hom_spkr.pdf')
plt.savefig(output_file)

plt.show()

#### Homer's lines in disguise

As a benchmark against which to measure Eudocia's selection of Homeric lines spoken in disguis, we can look at how often Homer himself uses disguised speakers. 

The Homer data comprises all lines included in direct speech in Homer. The Apologue of Odysseus, the multi-book tale of his wanderings narrated to Alcinous and Arete, is counted towards the total. Lines in embedded speeches are counted only for the "innermost" speaker: for example, when Odysseus reports the words of Circe, the are attributed to Circe and not Odysseus.

In [ ]:
homer_data_path = 'https://raw.githubusercontent.com/cwf2/homerocentones/main/data/homer_speech_lines.csv'
homer_df = pd.read_csv(homer_data_path)

# table: disguised or not
homer_disg_mask = homer_df['spkr'].str.contains('-')

homer_is_disg = homer_df.groupby(homer_disg_mask).agg(
    lines = ('line', 'count'),
)

display(homer_is_disg)

In [ ]:
# pie chart
fig, ax  = plt.subplots()

wedges, texts, autotexts = ax.pie(
    x = homer_is_disg['lines'],
    labels = ['no disguise', 'disguised'],
    labeldistance = None,
    counterclock = False,
    autopct = '%1.f%%',
)
ax.set_title('Homer’s use of disguised speakers')
ax.legend(loc='center left', bbox_to_anchor=(0.9, 0, 1, 0.5), reverse=True)
plt.setp(autotexts, size=14, weight="bold", color="white")

# save figure as PDF
output_file = os.path.join('output', 'fig_hom_is_disg.pdf')
plt.savefig(output_file)

plt.show()

#### Homer's disguised characters

How do these lines spoken in disguise break down amongst the characters? 

In [ ]:
homer_by_disg = homer_df.loc[homer_disg_mask].groupby('spkr').agg(
    lines = ('spkr', 'count'),
)

display(homer_by_disg)

#### Eudocia's use of lines in disguise

So how does Eudocia use the Homeric pool of disguised speech lines? Her ratio of disguised Homeric speakers to non-disguised speakers is very similar to the source material. This suggests that she does not favour or avoid Homer's disguised speakers in general.

In [ ]:
# table: disguised or not
disg_mask = df['hom_spkr'].str.contains('-')

is_disg = df.groupby(disg_mask).agg(
    lines = ('coefficient.1', 'sum'),
)

display(is_disg)

In [ ]:
# pie chart
fig, ax  = plt.subplots()

wedges, texts, autotexts = ax.pie(
    x = is_disg['lines'],
    labels = ['no disguise', 'disguised'],
    labeldistance = None,
    counterclock = False,
    autopct = '%1.f%%',
)
ax.set_title('Eudocia’s use of disguised Homeric speakers')
ax.legend(loc='center left', bbox_to_anchor=(0.9, 0, 1, 0.5), reverse=True)
plt.setp(autotexts, size=14, weight="bold", color="white")

# save figure as PDF
output_file = os.path.join('output', 'fig_is_disg.pdf')
plt.savefig(output_file)

plt.show()

#### Eudocia's selection of disguised characters

Which of Homer's disguised characters does Eudocia use in her speeches?

In [ ]:
# table: disguised characters 
by_disg = df.loc[disg_mask].groupby('hom_spkr').agg(
    lines = ('coefficient.1', 'sum'),
).sort_index()

order = [
    'Odysseus-beggar',
    'Odysseus-stranger',
    'Athena-Mentor',
    'Athena-Mentes',
    'Athena-maiden',
    'Athena-herald',
    'Athena-companion of Nausicaa',
    'Hermes-young man',
    'Poseidon-Calchas',
    'dream-Iphthime',
    'dream-Nestor',]
by_disg = by_disg.loc[order, :]

display(by_disg)

In [ ]:
# set up colors -- same color different alpha for disguised chars
colors = ['tab:blue', 'tab:blue', 'tab:orange', 'tab:orange', 'tab:orange', 'tab:orange', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:purple']
alphas = [0.5, 0.8, 0.2, 0.4, 0.6, 0.8, 1, 0.6, 0.6, 0.5, 0.8]
rgbas = []
for name, a in zip(colors, alphas):
    r,g,b = matplotlib.colors.to_rgb(name)
    rgbas.append((r,g,b,a))

# draw pie chart
fig, ax  = plt.subplots(figsize=(8,5))

wedges, texts, autotexts = ax.pie(
    x = by_disg['lines'],
    labels = by_disg.index,
    colors = rgbas,
    counterclock = True,
    startangle = 71,
    autopct = '%1.f%%',
    pctdistance = 0.7,
)

ax.set_title('Eudocia’s use of disguised Homeric speakers')
plt.setp(autotexts, size=10, weight="bold")

# save figure as PDF
output_file = os.path.join('output', 'fig_by_disg.pdf')
plt.savefig(output_file)

plt.show()

## Deceptive speech in Eudocia

The following graphs are based on reader judgments of the deceptiveness of lines in their original Homeric context. Each individual source line was evaluated, and then the source speech as a whole was also evaluated. Thus, a deceptive speech might contain many individual lines that are not in themselves deceptive, while a speech that was largely not deceptive might contain one or two lines that were nevertheless untrue. This work is ongoing, and a portion of the lines remain unclassified.

The categories used are:
- **None**: no deception
- **Possible**: possible or slight deception
- **Strong**: strong deception
- **unclassified**: no data

In [ ]:
# this function formats details of a speech suitable for a title
def getEudociaSpeechTitle(seq):
    '''Generate a string representing the speech'''

    spkr, addr, l_fi = df.groupby('eud_seq').agg('first').loc[seq, ['eud_spkr', 'eud_addr_all', 'eud_line']]
    l_la = df.groupby('eud_seq').agg('last').loc[seq, 'eud_line']

    return f'Eud. Hom. {l_fi}-{l_la}: {spkr} to {addr}'


In [ ]:
# this custom function is used to plot speeches by their position in Eudocia
def plotEudociaSpeech(seq, aspect=1.4, output_file=None):
    '''Generate a plot showing the source speaker and deceptiveness
       for each line of a speech in Eudocia'''
    
    # select colours
    pal = [
        (0.3400, 0.8288, 0.8600), # cyan
        (0.3127, 0.6929, 0.1924), # green
        (0.8288, 0.8600, 0.3400), # yellow
        (0.8600, 0.3712, 0.3400), # red
    ]

    # draw plot
    ax = sns.catplot(
        data = df.loc[df['eud_seq']==seq], 
        x = 'eud_line', 
        y = 'hom_spkr', 
        hue = 'dec_line', 
        kind = 'swarm',
        palette = pal,
        aspect = aspect,
    )

    # add axis labels, title
    ax.set(
        xlabel = 'Eudocia line',
        ylabel = 'Homeric speaker',
        title = getEudociaSpeechTitle(seq),
    )

    # position legend
    sns.move_legend(
        ax, "upper right",
        title = 'Deception',
        frameon = True,
        bbox_to_anchor=(1, 1)
    )

    # optional: save to file
    if filename is not None:
        plt.savefig(filename)

    # display
    plt.show()

#### Example 1: The speech of the Snake to Eve

In [ ]:
filename = os.path.join('output', 'fig_ex_1.pdf')
plotEudociaSpeech(1, aspect=1.6, output_file=filename)

#### Example 3: The happy message of Easter

In [ ]:
filename = os.path.join('output', 'fig_ex_3.pdf')
plotEudociaSpeech(83, aspect=1.6, output_file=filename)

#### Example 4: The annointing of Jesus

In [ ]:
filename = os.path.join('output', 'fig_ex_4.pdf')
plotEudociaSpeech(47, output_file=filename)

### Interactive

Use the interactive pane below to explore other speeches of Eudocia.

In [ ]:
dropdown = [(getEudociaSpeechTitle(seq), seq) for seq in df['eud_seq'].unique()]
interactive_plot = interactive(plotEudociaSpeech, 
    seq=dropdown, aspect=fixed(1.4), output_file=fixed(None))
display(interactive_plot)